<h3>Testing with only one input</h3>

In [1]:
import requests
import pandas as pd

url = 'http://127.0.0.1:8000/predict'

test_payload = {
    "vibration_level": 0.446974,
    "temperature_C": 72.827548,
    "pressure_PSI": 1.820521,
    "flow_rate_m3h": 0,
    "hour": 22,
    "day_of_week": 4,
    "week_of_month": 3,
    "vibration_level_rolling_mean": 0.497611,
    "temperature_C_rolling_mean": 75.418994,
    "pressure_PSI_rolling_mean": 96.302199,
    "flow_rate_m3h_rolling_mean": 44.649508,
    "vibration_trend": 0.052896,
    "temp_trend": 2.721190,
    "pressure_trend": 13.725931,
    "flow_trend": -12.579165
}

response = requests.post(url, json=test_payload)

# Check if the request was successful
if response.status_code == 200:
    print(response.text)
else:
    print(f"Error: {response.status_code} - {response.text}")

{"prediction":1,"prediction_proba":0.9595866799354553}


<h3>Batch Testing</h3>

In [2]:
df_test = pd.read_csv('test_data_api.csv')  # loading the batch test data from notebook
df_test.drop(columns=['Unnamed: 0', 'failure'], inplace=True)

results = []

for index, row in df_test.iterrows():
    timestamp = row['timestamp']

    # Create the payload by selecting all columns except 'timestamp'
    payload = row.drop(labels=['timestamp']).to_dict()
    response = requests.post(url, json=payload)

    prediction = None
    prediction_proba = None

    if response.status_code == 200:
        try:
            response_json = response.json()
            prediction = response_json.get("prediction")
            prediction_proba = response_json.get("prediction_proba")
        except ValueError:
            prediction = "Error: Invalid JSON"
            prediction_proba = "Error: Invalid JSON"
    else:
        prediction = f"Error: {response.status_code}"
        prediction_proba = f"Error: {response.status_code}"

    result = row.to_dict()
    result['prediction'] = prediction
    result['prediction_proba'] = prediction_proba
    results.append(result)

df_results = pd.DataFrame(results)

df_results

,timestamp,vibration_level,temperature_C,pressure_PSI,flow_rate_m3h,hour,day_of_week,week_of_month,vibration_level_rolling_mean,temperature_C_rolling_mean,pressure_PSI_rolling_mean,flow_rate_m3h_rolling_mean,vibration_trend,temp_trend,pressure_trend,flow_trend,prediction,prediction_proba
0,2024-01-01 05:00:00,0.476586,71.967427,102.132937,43.322592,5,0,1,0.531283,71.429036,94.149558,49.834769,-0.073085,-5.029350,8.884720,2.861630,0,0.000230
1,2024-01-01 06:00:00,0.657921,74.475966,100.012055,52.430181,6,0,1,0.565633,71.399596,94.441006,51.181190,0.171748,-0.147202,1.457241,6.732107,0,0.000691
2,2024-01-01 07:00:00,0.576743,73.175859,91.829114,42.263480,7,0,1,0.568028,71.975137,94.391669,50.047492,0.011975,2.877707,-0.246687,-5.668492,0,0.000387
3,2024-01-01 08:00:00,0.453053,75.247764,106.592457,55.413455,8,0,1,0.528178,73.671626,96.326083,49.242495,-0.199250,8.482447,9.672072,-4.024983,0,0.000840
4,2024-01-01 09:00:00,0.554256,67.323824,109.375701,47.644377,9,0,1,0.543712,72.438168,101.988453,48.214817,0.077671,-6.167293,28.311848,-5.138389,0,0.003593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,2024-02-11 11:00:00,0.471890,75.350751,100.774805,50.142288,11,6,2,0.414928,74.073107,99.175017,46.267722,-0.048948,1.348704,-14.389135,8.588203,0,0.000272
991,2024-02-11 12:00:00,0.679769,69.867394,102.577525,39.610941,12,6,2,0.491716,73.292294,98.486285,44.661174,0.383942,-3.904063,-3.443658,-8.032740,1,0.967985
992,2024-02-11 13:00:00,0.564084,65.590627,87.582394,48.398511,13,6,2,0.509476,71.221506,95.858690,46.316364,0.088802,-10.353940,-13.137974,8.275950,0,0.001180
993,2024-02-11 14:00:00,0.442882,69.184665,103.341764,58.216891,14,6,2,0.511693,70.350136,96.951461,47.208643,0.011081,-4.356854,5.463854,4.461394,0,0.001946
